In [3]:
import os
import csv
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import keras
import PIL
import struct
import cv2
import math
import pandas as pd

import tensorflow as tf
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
 
# %matplotlib inline

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path = '/content/drive/My Drive/'

In [8]:
file_name = path+'pd_file.csv'
file = pd.read_csv(file_name)
dct_filename_coords = {}

file_names = file.image_name.unique()
for name in file_names:
  dct_filename_coords[name] = []
  sub_df = file.loc[file.image_name == name]
  n = sub_df.shape[0]
  for j in range(n):
    temp = [sub_df.iloc[j]['x1'], sub_df.iloc[j]['y1'], sub_df.iloc[j]['x2'], sub_df.iloc[j]['y2']]
    dct_filename_coords[name].append(temp)

In [9]:
grid_height = 25
grid_width = 31

img_height = 200
img_width = 250

In [10]:
lst = os.listdir(path+'Training pics/Pics')
print(lst)



imgs = []
outs = []

for name in lst:
  print(name)
  img = cv2.imread(path+'Training pics/Pics/' + name)
  height = img.shape[0]
  width = img.shape[1]
  out_arr = np.zeros((grid_height,grid_width, 5))
  coords = dct_filename_coords[name]
  x_unit_pixel = height/grid_height
  y_unit_pixel = width/grid_width
  
  for i in range(grid_height):
    for j in range(grid_width):
      centre_x = (i+0.5)*x_unit_pixel
      centre_y = (j+0.5)*y_unit_pixel
      for coord in coords:
        if (centre_x > coord[0]) and (centre_x < coord[3]) and (centre_y > coord[1]) and (centre_y < coord[3]):
          out_arr[i,j,0] = 1
          out_arr[i,j,1] = coord[0]/height
          out_arr[i,j,2] = coord[1]/width
          out_arr[i,j,3] = coord[2]/height
          out_arr[i,j,4] = coord[3]/width
  print(coords, out_arr)
  imgs.append(cv2.resize(img, (250,200)))
  outs.append(out_arr)

Streaming output truncated to the last 5000 lines.
  [1.00000000e+00 5.14403292e-04 5.59413580e-02 1.28086420e-01
   2.34953704e-01]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]]

 [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [1.00000000e+00 5.14403292e-04 5.59413580e-02 1.28086420e-01
   2.34953704e-01]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]]

 [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]
  [0.00000000e+00 0.00000000e+0

KeyError: ignored

In [11]:
imgs = np.asarray(imgs)
outs = np.asarray(outs)
print(imgs.shape, outs.shape)

(1243, 200, 250, 3) (1243, 25, 31, 5)


In [12]:
xTrain, xTest, yTrain, yTest = train_test_split(imgs, outs, test_size = 0.2, random_state = 0)
print(xTrain.shape, yTrain.shape)

(994, 200, 250, 3) (994, 25, 31, 5)


In [13]:

def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    for conv in convs:
        if count == (len(convs) - 2) and skip:
            skip_connection = x
        count += 1
        
        if conv['stride'] > 1: x = tf.keras.layers.ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
        x = tf.keras.layers.Conv2D(conv['filter'], 
                   conv['kernel'], 
                   strides=conv['stride'], 
                   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
                   name='conv_' + str(conv['layer_idx']), 
                   use_bias=False if conv['bnorm'] else True)(x)
        if conv['bnorm']: x = tf.keras.layers.BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']: x = tf.keras.layers.LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)

    return tf.keras.layers.add([skip_connection, x]) if skip else x


In [14]:

# creating the YOLO model
def make_yolov3_model():
    input_image = tf.keras.layers.Input(shape=(img_height,img_width, 3))

    # Layer  0 => 4
    x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])

    # Layer  5 => 8
    x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                        {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])

    # Layer  9 => 11
    x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])

    # Layer 12 => 15
    x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
                        {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
                        {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])

    # Layer 16 => 36
    for i in range(7):
        x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
                            {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
        
    skip_36 = x
        
    # Layer 37 => 40
    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])

    # Layer 41 => 61
    for i in range(7):
        x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
                            {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
        
    skip_61 = x
        
    # Layer 62 => 65
    x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])

    # Layer 66 => 74
    for i in range(3):
        x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
                            {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
        
    # Layer 75 => 79
    x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)

    # Layer 80 => 82
    yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
                              {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)

    # Layer 83 => 86
    x = _conv_block(x, [{'filter': 5, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
    
    x = tf.keras.layers.Activation('sigmoid')(x)
   

    
    model = tf.keras.Model(input_image, outputs = x)

    return model


In [15]:
yolov3 = make_yolov3_model()
yolov3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 250, 3) 0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 200, 250, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, 200, 250, 32) 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, 200, 250, 32) 0           bnorm_0[0][0]                    
______________________________________________________________________________________________

In [16]:
def custom_loss(y_true, y_pred):
  prob_of_object_true = y_true[:,:,:,0:1]
  prob_of_object_pred = y_pred[:,:,:,0:1]
  pred_loss = tf.keras.losses.BinaryCrossentropy()(prob_of_object_true, prob_of_object_pred)
  
  selected_pred = tf.math.multiply(prob_of_object_true, y_pred)
  
  selected_pred_coord = selected_pred[:,:,:,1:5]
  mse_loss = keras.losses.mse(selected_pred_coord, y_true[:,:,:,1:5])
  loss = pred_loss + mse_loss
  return loss

In [19]:
optimizer = tf.keras.optimizers.Adam(0.001)

yolov3.compile(
    optimizer=optimizer, loss=custom_loss, metrics=custom_loss
)

In [25]:
yolov3.fit(
    x=xTrain, y=yTrain, batch_size=10, epochs=500, verbose='auto'
)
yolov3.save(path+'/yolov3.hdf5')

Epoch 1/500
100/100 [==============================] - 3413s 34s/step - loss: 0.6849 - custom_loss: 0.6849
Epoch 2/500
 12/100 [==>...........................] - ETA: 49:48 - loss: 0.6661 - custom_loss: 0.6661

KeyboardInterrupt: ignored

In [ ]:
yolov3 = tf.keras.models.load_model(path+'/yolov3.hdf5')

In [ ]:
i = 11
shape = xTest[i, :, :, :].shape
imge = np.zeros((1, shape[0], shape[1], shape[2]))
np.copyto(imge, xTest[i:i+ 1, :, :, :])

height = imge[0].shape[0]
width = imge[0].shape[1]
img = cv2.resize(imge[0], (250,200))
x_unit_pixel = height/grid_height
y_unit_pixel = width/grid_width
img = np.expand_dims(imge[0], axis = 0)

pred = yolov3(img)
pred = pred[0,:,:,:].numpy()
ref = 0.5


coords = []
for i in range(pred.shape[0]):
  for j in range(pred.shape[1]):
    conf = pred[i,j,0]
    if conf <= ref :
      pass
    else:
      coords.append([int(pred[i,j,1]*height), int(pred[i,j,2]*width), int(pred[i,j,3]*height), int(pred[i,j,4]*width)])

img = imge[0]
for coord in coords:
  img = cv2.rectangle(img, (pred[i,j,1], pred[i,j,2]), (pred[i,j,3], pred[i,j,4]),  (0,255,0), 5)
cv2_imshow(img)        
img.shape